In [1]:
import pandas as pd
import plotly.express as px
import numpy as np

In [2]:
def preprocess_data(path = '../timesheet_dash_data/RESOURCE SUMMARY.xlsx'):
    wb = pd.read_excel(path,'Sheet1',header=1)

    correspondences = {}

    nonworking = ['10th Day','9th Day','8th day','All Saints Day','Annual Leave','Ascension Day','Easter Monday','National Day',
    'Other Leave','Sick Leave','Special Leave granted by the DG','Whit Monday']

    non_audit = ['Administrative Matters & Support','Covid-19 Sanitary Situation','GC and EXB (incl. Annual report)',
        'HR Management & Recruitment', 'IOS Management - Ad-hoc Requests','IOS Team Meetings', 'JIU Coordination',
        'OAC support, preparation and meetings','Participation to UNESCO Working Groups or Task Forces',
        'Policy or Administrative Manual Item Review','Support to Investigation Unit',
        'Trainings or Workshops']

    audit = ['Audit-Ad-hoc request / Advisory','Audit-Annual Planning', 'Audit-QAIP (incl. TeamMate+ Migration)']

    reco = ['Audit-Recommendation Follow-up']

    correspondences['Nonworking'] = nonworking
    correspondences['Non audit time'] = non_audit
    correspondences['Audit Time'] = audit
    correspondences['Reco Follow-up Time'] = reco

    # Insert correspondences for the different Time Categories
    wb['Reporting categories'] = 'Audit Time'
    for i in correspondences: 
        wb['Reporting categories'].mask(wb['Time Category'].isin(correspondences[i]),i,inplace=True)

    # Make Phase 5 -- Recommendation followups be repored as Reco Follow up time
    wb['Reporting categories'].mask(wb['Phase'].astype('string').str.startswith('5'),'Reco Follow-up Time',inplace=True)

    # Remove IOS/... prefix from Assignment name
    prefix_removed = wb['Assignment Name'].str.extract(r'(^IOS/[a-zA-Z0-9_\./]+)-([a-zA-Z0-9_\' -]+)')[1]
    idxs = ~wb['Assignment Name'].str.startswith('IOS/').fillna(False)
    prefix_removed[idxs] = wb[idxs]['Assignment Name']
    wb['Assignment Name'] = prefix_removed


    return wb


In [3]:
wb = preprocess_data()

In [4]:
names = wb['Resource'].dropna().unique()